In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
with open('/content/drive/My Drive/YTP/dialog-ptt.txt') as f:
    dialog=f.readlines()
n=len(dialog)
print(n)

774114


In [ ]:
#@title
# import numpy as np
# temp = np.load('/content/drive/My Drive/YTP/ptt_training.npz',allow_pickle=True)
# x_train,y_train=temp['x'],temp['y']
# # x_train=np.load('/content/drive/My Drive/YTP/ptt_stop_training_x.npy',allow_pickle=True)
# # y_train=np.load('/content/drive/My Drive/YTP/ptt_stop_training_y.npy',allow_pickle=True)
# wvs=250
# n=len(x_train)
# v_start=x_train[0][0]
# v_end=x_train[0][-1]

In [ ]:
#@title padding
wvs=250
n=len(x_train)
v_start=x_train[0][0]
v_end=x_train[0][-1]
maxx,maxy=0,0
for line in x_train:
    maxx=max(maxx,len(line))
for line in y_train:
    maxy=max(maxy,len(line))
maxx,maxy=50,30
for i in range(n):
    while len(x_train[i])<maxx:
        x_train[i].append(v_end)
    while len(y_train[i])<maxy:
        y_train[i].append(v_end)
    if len(x_train[i])!=maxx:
        print(i,'x',len(x_train[i]))
    if len(y_train[i])!=maxy:
        print(i,'y',len(y_train[i]))
print(x_train.shape,y_train.shape)

# word2vec

## data generator

In [ ]:
#@title
batch_size=320
def data_gen(rang=None):
    if rang==None:
        rang=n
    while True:
        ind=np.random.randint(0,n,size=(batch_size))
        x1,x2,y=[],[],[]
        for i in ind:
            x1.append(x_train[i])
            x2.append(y_train[i])
            y.append(np.append(y_train[i][1:],[v_end],axis=0))
        yield ([np.array(x1),np.array(x2)],np.array(y))
t=data_gen()

In [ ]:
d=next(t)
print(d[0][0].shape)
print(d[0][1].shape)
print(d[1].shape)
print([int(np.array_equal(i,v_end)) for i in d[0][1][0]])
print([int(np.array_equal(i,v_end)) for i in d[1][0]])
print([int(np.array_equal(i,v_end)) for i in d[0][1][1]])
print([int(np.array_equal(i,v_end)) for i in d[1][1]])
print([int(np.array_equal(i,v_end)) for i in d[0][1][2]])
print([int(np.array_equal(i,v_end)) for i in d[1][2]])

## [example](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html)

In [ ]:
from keras.layers import Dense,LSTM,Activation,RepeatVector,Input,Masking
from keras.models import Model
from keras.optimizers import Adam
import numpy as np
maxx,maxy=40,40
v_end=np.zeros((250,))
v_start=np.ones((250,))
wvs=250
input_layer1 = Input((maxx,wvs))
x = Masking(v_end)(input_layer1)
x = LSTM(1024,return_sequences=True)(x)
x,sh,sc = LSTM(1024,return_state=True)(x)

input_layer2 = Input((maxy,wvs))
x = Masking(v_end)(input_layer2)
x = LSTM(1024,return_sequences=True)(x,initial_state=[sh,sc])
output_layer = LSTM(wvs,return_sequences=True)(x)
model = Model([input_layer1,input_layer2],output_layer)
model.compile(loss='mse',optimizer=Adam())
model.summary()

In [ ]:
model.summary()

In [ ]:
from keras.models import load_model
model=load_model('/content/drive/My Drive/YTP/instructions.h5')

In [ ]:
model.summary()

In [ ]:
gen=data_gen()
model.fit_generator(gen,epochs=50,steps_per_epoch=100)

In [ ]:
model.evaluate_generator(data_gen(),steps=1)

0.49007850885391235

## sampling

In [ ]:
from gensim.models import Word2Vec
import jieba
temp = Word2Vec.load('/content/drive/My Drive/YTP/ptt+wiki.model')
w2v=temp.wv
v_start=w2v['<start>']
v_end=w2v['<end>']
jieba.set_dictionary('/content/drive/My Drive/YTP/dict.txt')
stopword_set = set()
# with open('/content/drive/My Drive/stopwords.txt','r',encoding='utf-8') as stopwords:
#     for stopword in stopwords:
#         stopword_set.add(stopword.strip('\n'))

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
def v2w(vec):
    text=[]
    for i in range(len(vec)):
        text.append(w2v.most_similar(positive=[vec[i]])[0][0])
    return text
def sample(question,model):
    words=jieba.cut(question, cut_all=False)
    sentence=[v_start]
    for word in words:
        if word.isspace():
            continue
        if word not in stopword_set:
            if word in w2v.vocab:
                sentence.append(w2v[word])
    while len(sentence)<maxx:
        sentence.append(v_end)
    if len(sentence)!=maxx:
        print('too long')
        return
    ans=[v_start]
    state=[v_end for _ in range(maxy)]
    state[0]=v_start
    for i in range(0,maxy-1):
        # print(v2w(state))
        # print('------')
        pre=model.predict([[sentence],[state]])[0]
        state[i+1]=pre[i]
        ans.append(pre[i])
    text=[]
    for i in range(maxy):
        text.append(w2v.most_similar(positive=[ans[i]])[0][0])
    print(text)
    return ans
ans=sample('求掛',model)

In [ ]:
from keras.models import load_model
model=load_model('/content/drive/My Drive/YTP/instructions.h5')

In [ ]:
print(model.output_shape)

(None, None, 250)


## cool

In [ ]:
from keras.layers import Dense,LSTM,Input
from keras.models import Model
import numpy as np

wvs=250
latent_dim = 512
encoder_inputs = Input(shape=(None, wvs))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, wvs))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(wvs)
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='cosine_proximity'')

In [ ]:
gen=data_gen(20)
model.fit_generator(gen,epochs=50,steps_per_epoch=100)

In [ ]:
model.save('/content/drive/My Drive/instructions.h5')

## sample

In [ ]:
#@title
from gensim.models import Word2Vec
import jieba
temp = Word2Vec.load('/content/drive/My Drive/ptt+wiki.model')
w2v=temp.wv
v_start=w2v['<start>']
v_end=w2v['<end>']
jieba.set_dictionary('/content/drive/My Drive/dict.txt')
stopword_set = set()
with open('/content/drive/My Drive/stopwords.txt','r',encoding='utf-8') as stopwords:
    for stopword in stopwords:
        stopword_set.add(stopword.strip('\n'))

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.

def decode_sequence(question):
    words=jieba.cut(question, cut_all=False)
    sentence=[v_start]
    for word in words:
        if word.isspace():
            continue
        if word not in stopword_set:
            if word in w2v.vocab:
                sentence.append(w2v[word])
    while len(sentence)<maxx:
        sentence.append(v_end)
    if len(sentence)!=maxx:
        print('too long for the question')
        return
    input_seq=np.array([sentence])
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, wvs))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = v_start

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = '<start>'
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_vector = output_tokens
        sampled_char = w2v.most_similar(positive=[sampled_token_vector[0][0]])[0][0]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '<end>' or
           len(decoded_sentence) > maxy):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, wvs))
        target_seq[0, 0] = sampled_token_vector

        # Update states
        states_value = [h, c]

    return decoded_sentence

print(decode_sequence('機車推出uber或計程機車會怎樣ㄐㄣ'))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


<start>?,,,,,??<end>


In [ ]:
#@title
with open('/content/drive/My Drive/Gossiping-QA-Dataset-2_0.csv') as f:
    for i,j in enumerate(f.readlines()):
        if i not in range(0,30):
            continue
        print(j,end='')

## not teacher forcing

In [ ]:
batch_size=320
def data_gen(rang=None):
    if rang==None:
        rang=n
    while True:
        ind = np.random.randint(0,rang,batch_size)
        x2 = np.zeros((batch_size, 1, wvs))
        x2[:, 0] = v_start
        x1=[]
        y=[]
        for i in ind:
            x1.append(np.array(x_train[i]))
            y.append(np.array(y_train[i][1:]))
        # y=np.array(y_train[ind][1:])
        yield [np.array(x1),x2],np.array(y)

In [ ]:
t=data_gen()
a=next(t)
print(a[0][0].shape)
print(a[0][1].shape)
print(a[1].shape)

(320, 40, 250)
(320, 1, 250)
(320, 39, 250)


In [ ]:
from keras.layers import Lambda,Input,Dense,LSTM
from keras.models import Model
from keras import backend as K

latent_dim = 512
wvs=250
maxy=40
# The first part is unchanged
encoder_inputs = Input(shape=(None, wvs))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
states = [state_h, state_c]

# Set up the decoder, which will only process one timestep at a time.
decoder_inputs = Input(shape=(1, wvs))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_dense = Dense(wvs)

all_outputs = []
inputs = decoder_inputs
for _ in range(maxy-1):
    # Run the decoder on one timestep
    outputs, state_h, state_c = decoder_lstm(inputs,
                                             initial_state=states)
    outputs = decoder_dense(outputs)
    # Store the current prediction (we will concatenate all predictions later)
    all_outputs.append(outputs)
    # Reinject the outputs as inputs for the next loop iteration
    # as well as update the states
    inputs = outputs
    states = [state_h, state_c]

# Concatenate all predictions
decoder_outputs = Lambda(lambda x: K.concatenate(x, axis=1))(all_outputs)

# Define and compile model as previously
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='Adam', loss='cosine_proximity')
model.summary()

In [ ]:
from keras.models import load_model
model = load_model('/content/drive/My Drive/YTP/not_teacher_forcing_version.h5')

In [ ]:
gen=data_gen(10)
train_model.fit_generator(gen,epochs=30,steps_per_epoch=100)

In [ ]:
model.save('/content/drive/My Drive/not_teacher_forcing_version.h5')

In [ ]:
from gensim.models import KeyedVectors
import jieba
w2v=KeyedVectors.load('/content/drive/My Drive/YTP/ptt+wiki.kv', mmap='r')
v_start=w2v['<start>']
v_end=w2v['<end>']
jieba.set_dictionary('/content/drive/My Drive/YTP/dict.txt')
stopword_set = set()
# with open('/content/drive/My Drive/stopwords.txt','r',encoding='utf-8') as stopwords:
#     for stopword in stopwords:
#         stopword_set.add(stopword.strip('\n'))
def v2w(vec):
    text=[]
    for i in range(len(vec)):
        text.append(w2v.most_similar(positive=[vec[i]])[0][0])
    return text
def compose(texts):
    s = ''
    for i in texts:
        if i=='<start>':
            continue
        elif i=='<end>':
            return s
        else:
            s=s+i
    return s
def str2vs(string):
    words=jieba.cut(string, cut_all=False)
    sentence=[v_start]
    for word in words:
        if word.isspace():
            continue
        if word not in stopword_set:
            if word in w2v.vocab:
                sentence.append(w2v[word])
    while len(sentence)<maxx:
        sentence.append(v_end)
    return sentence

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
print(compose(str2vs('早安')))

In [ ]:
t=data_gen(10)
a=next(t)
ans=model.predict(a[0])
for i in range(10):
    print('Q:',compose(v2w(a[0][0][i])))
    print('A:',compose(v2w(ans[i])))
    print('T:',compose(v2w(a[1][i])))
    print()

# LSTM

In [ ]:
#@title model
# new
from keras.layers import Lambda,Input,Dense,LSTM
from keras.models import Model
from keras.optimizers import Adam
import keras.backend as K

latent_dim = 512
wvs=250
maxx,maxy=50,30

IN=Input((maxx,wvs))
OUT=IN
OUT=LSTM(latent_dim,return_sequences=True)(OUT)
OUT=LSTM(latent_dim,return_sequences=True)(OUT)
OUT=LSTM(latent_dim,return_sequences=True)(OUT)
OUT,state_h,state_c=LSTM(latent_dim,return_state=True)(OUT)
encoder=Model(IN,[state_h,state_c])

depth=2
IN=[Input(shape=(1, wvs))]
for i in range(depth-1):
    IN+=[Input(shape=(latent_dim,)),Input(shape=(latent_dim,))]
IN+=[Input(shape=(wvs,)),Input(shape=(wvs,))]
OUT=[None]
out=IN[0]
for i in range(depth-1):
    out,state_h,state_c=LSTM(latent_dim,return_sequences=True,return_state=True)(out,initial_state=IN[i*2+1:i*2+3])
    OUT+=[state_h,state_c]
out,state_h,state_c=LSTM(wvs,return_sequences=True,return_state=True)(out,initial_state=IN[-2:])
OUT+=[state_h,state_c]
OUT[0]=out
decoder=Model(IN,OUT)

IN=[Input(shape=(1, wvs)),Input(shape=(latent_dim,)),Input(shape=(latent_dim,))]
OUT=[None]
out=IN[0]
for i in range(depth-1):
    if i==0:
        out,state_h,state_c=LSTM(latent_dim,return_sequences=True,return_state=True)(out,initial_state=IN[1:3])
    else:
        out,state_h,state_c=LSTM(latent_dim,return_sequences=True,return_state=True)(out)
    OUT+=[state_h,state_c]
out,state_h,state_c=LSTM(wvs,return_sequences=True,return_state=True)(out)
OUT+=[state_h,state_c]
OUT[0]=out
decoder_init=Model(IN,OUT)

IN=[Input((maxx,wvs)),Input((1,wvs))]
OUT=[]
out=encoder(IN[0])
for i in range(maxy-1):
    if i==0:
        temp=decoder_init([IN[1]]+out)
    else:
        temp=decoder(out)
    OUT.append(temp[0])
    out=temp

OUT=Lambda(lambda x: K.concatenate(x, axis=1))(OUT)
train_model=Model(IN,OUT)

In [ ]:
train_model.compile(optimizer=Adam(),loss='cosine_proximity')

In [ ]:
import numpy as np
import jieba
from gensim.models import KeyedVectors
w2v=KeyedVectors.load('/content/drive/My Drive/YTP/ptt+wiki-stop.kv',mmap='r')
v_start=w2v['<start>']
v_end=w2v['<end>']
jieba.set_dictionary('/content/drive/My Drive/YTP/dict.txt')
batch_size=32
def to_vec(text):
    text=text.strip('\n')
    words=jieba.cut(text,cut_all=False)
    res=[]
    for word in words:
        if word in w2v.vocab:
            res.append(w2v[word])
    return res

def data_gen(rang=None):
    if rang==None:
        rang=n
    while True:
        ind=np.random.randint(0,rang,size=batch_size)
        # print(ind)
        x1,x2,y=[],[],[]
        for i in ind:
            q,a=dialog[i].split('\t')
            x1.append([v_start]+to_vec(q)+[v_end])
            x2.append([v_start])
            y.append(to_vec(a)+[v_end])
        for i in x1:
            while len(i)<maxx:
                i.append(v_end)
        for i in y:
            while len(i)<maxy-1:
                i.append(v_end)
        yield ([np.array(x1),np.array(x2)],np.array(y))
t=data_gen()

In [ ]:
gen=data_gen(10)
history=train_model.fit_generator(gen,epochs=10,steps_per_epoch=100,verbose=1)

In [ ]:
train_model.evaluate_generator(gen,steps=10)

In [ ]:
train_model.save('/content/drive/My Drive/YTP/deep-model.h5')

In [ ]:
train_model.summary()

In [ ]:
from gensim.models import KeyedVectors
import jieba
w2v=KeyedVectors.load('/content/drive/My Drive/YTP/word2vec_with_stop.kv',mmap='r')
v_start=w2v['<start>']
v_end=w2v['<end>']
jieba.set_dictionary('/content/drive/My Drive/YTP/dict.txt')

In [ ]:
def sample(question):
    def v2w(x):
        return w2v.most_similar(positive=[x])[0][0]
    words=jieba.cut(question,cut_all=False)
    sentence=[v_start]
    for word in words:
        if word.isspace():
            continue
        if word in w2v.vocab:
            sentence.append(w2v[word])
    while len(sentence)<maxx:
        sentence.append(v_end)
    if len(sentence)!=maxx:
        print('too long for the question')
        return None
    ans=[v_start]
    text=['<start>']
    pre=train_model.predict([[sentence],[[v_start]]])
    for i in pre[0]:
        text.append(v2w(i))
    # pre=encoder.predict(np.array([sentence]))
    # IN=[[[v_start]]]+[[i[0]] for i in pre]
    # IN=decoder_init.predict(IN)
    # ans.append(IN[0][0][0])
    # text.append(v2w(IN[0][0][0]))
    # for i in range(maxy-2):
    #     IN=decoder.predict(IN)
    #     ans.append(IN[0][0][0])
    #     text.append(v2w(IN[0][0][0]))
    #     if text[-1]=='<end>':
    #         break
    return ' '.join(text)

In [ ]:
ans=sample('為什麼達摩祖師傳那麼好看?')
print(ans)

In [ ]:
train_model.summary()

In [ ]:
for i in range(20):
    print(dialog[i])

# teacher forcing

In [ ]:
#@title model
# new
from keras.layers import Lambda,Input,Dense,LSTM,Dropout,TimeDistributed
from keras.models import Model
from keras.optimizers import Adam,RMSprop
import keras.backend as K

latent_dim = 1024
wvs=250
maxx,maxy=30,30
rate=0.3

IN=Input((maxx,wvs))
OUT=IN
OUT=Dropout(rate)(OUT)
OUT=TimeDistributed(Dense(latent_dim))(OUT)
OUT=Dropout(rate)(OUT)
OUT,sh,sc=LSTM(latent_dim,return_state=True)(OUT)

yIN=Input((maxy,wvs))
OUT=LSTM(latent_dim,return_sequences=True)(yIN,initial_state=[sh,sc])
OUT=Dropout(rate)(OUT)
OUT=TimeDistributed(Dense(wvs))(OUT)

model=Model([IN,yIN],OUT)

In [ ]:
model.compile(optimizer=RMSprop(),loss='cosine_proximity')

In [ ]:
import numpy as np
import jieba
from gensim.models import KeyedVectors
w2v=KeyedVectors.load('/content/drive/My Drive/YTP/ptt+wiki.kv',mmap='r')
v_start=w2v['<start>']
v_end=w2v['<end>']
jieba.set_dictionary('/content/drive/My Drive/YTP/dict.txt')
stop_set=set()
with open('/content/drive/My Drive/YTP/stopwords.txt','r',encoding='utf-8') as stopwords:
    for stopword in stopwords:
        stop_set.add(stopword.strip('\n'))
batch_size=32
def to_vec(text):
    text=text.strip('\n')
    words=jieba.cut(text,cut_all=False)
    res=[]
    for word in words:
        if word in w2v.vocab and word not in stop_set:
            res.append(w2v[word])
    return res

def data_gen(rang=None):
    if rang==None:
        rang=n
    while True:
        ind=np.random.randint(0,rang,size=batch_size)
        # print(ind)
        x1,x2,y=[],[],[]
        for i in ind:
            q,a=dialog[i].split('\t')
            q,a=to_vec(q),to_vec(a)
            x1.append([v_start]+q+[v_end])
            x2.append([v_start]+a+[v_end])
            y.append(a+[v_end])
        for i in x1:
            while len(i)<maxx:
                i.append(v_end)
        for i in x2:
            while len(i)<maxy:
                i.append(v_end)
        for i in y:
            while len(i)<maxy:
                i.append(v_end)
        yield ([np.array(x1),np.array(x2)],np.array(y))
t=data_gen()

In [ ]:
from keras.callbacks import LambdaCallback
datas=1000
gen=data_gen(datas)
def sample(epoch,log):
    if (epoch+1)%5!=0:
        return
    for _ in range(3):
        i=np.random.randint(datas)
        print('============')
        print('Q:',dialog[i].split('\t')[0])
        print('A:',dialog[i].split('\t')[1],end='')
        print('a:',predict(dialog[i].split('\t')[0],model))
history=model.fit_generator(gen,epochs=200,steps_per_epoch=100,verbose=1,
                            callbacks=[LambdaCallback(on_epoch_end=sample)])

In [ ]:
model.save('/content/drive/My Drive/YTP/simple-model.h5')

In [ ]:
from keras.models import load_model
model=load_model('/content/drive/My Drive/YTP/simple-model.h5')

In [ ]:
model.summary(line_length=300)

In [ ]:
maxx,maxy=30,30
def v2w(x):
    return w2v.most_similar(positive=[x])[0][0]
def predict(question,model):
    qusetion=to_vec(question)
    while len(qusetion)<maxx:
        qusetion.append(v_end)
    ans=[v_end for _ in range(maxy)]
    ans[0]=v_start
    for i in range(1,maxy):
        ans[i]=model.predict([[qusetion],[ans]])[0][i-1]
    text=[v2w(i) for i in ans]
    return ' '.join(text)
print(predict('',model))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


<start> 反正 無腦 ? ? <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>


In [ ]:
def get_response(q):
    a=predict(q,model)
    return a.replace('<start> ','').replace(' <end>','')

In [ ]:
for q in questions:
    print('=====================')
    print(q)
    print(predict(q,model))

In [ ]:
while True:
    print('Q: ')
    q=input()
    if q=='exit':
        break
    print('A: '+get_response(q))
    print('==================')

Q: 
統一有違反公平交易法嗎？
A: 左打 味全 開單 很爽 ,
Q: 
國語女歌手誰排第一？
A: 阿妹 人氣 媽的
Q: 
有車有房的車跟房的標準是??
A: 兩百萬 ? ? ?
Q: 
exit
